In [4]:
# We start by importing SparkContext and creating a context
# which we will use in class

from pyspark import SparkContext
# sc.stop()
sc = SparkContext()

21/09/27 22:23:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Fundamendal RDD Functions

* In addition to `map`, `flatmap`, `filter` and `reduce`, Spark provides other useful RDD methods (transformations and action)

* The objective is to provide the set of functions that facilitate programming in the map-reduce paradigm.

* Some examples transformation are:
  * `distinct`: return an RDD comprised of the unique elements of the calling `RDD`
  * `union`: computes the union of two `RDD`s
  * `intersection`: computes the intersection of two `RDD`s
  * `foreach`: Takes a lambda function and applies to each element of the `RDD`
    * Like `map` but does not return values
  * `cartesian` returns the cartesian product between the calling `RDD` 
  * etc. 

* There are also various actions that can be applied to an `RDD`

  * `count`: returns the number of elements in the `RDD`
  * `sum`: returns the sum of of elements in the `RDD` 
    * assumes the values in the RDD are numric
  * `mean`: returns the sum of of elements in the `RDD` 
    * assumes the values in the RDD are numric
  * `stats`: computes and returns a tuple with the `count`, the `mean`, `stdev`, `max`, `min`
  * etc.
  

In [5]:
# Distinct example 
dataset_1 = sc.parallelize(["A", "B", "C", "A", "C", "D"])
dataset_1.distinct().collect()

['A', 'B', 'C', 'D']

In [6]:
# Union example

dataset_1 = sc.parallelize(["A", "B", "C"])
dataset_2 = sc.parallelize(["D", "E", "F"])

both_datasets = dataset_1.union(dataset_2)

both_datasets.collect()

['A', 'B', 'C', 'D', 'E', 'F']

In [7]:
# intersection example
dataset_1 = sc.parallelize(["A", "B", "C"])
dataset_2 = sc.parallelize(["N", "B", "C", "M"])
dataset_1.intersection(dataset_2).collect()

['B', 'C']

In [8]:
# foreach example
dataset_1.foreach(lambda x: print(x))

A
B
C


In [9]:
# cartesian example

dataset_1 = sc.parallelize(["A", "B"])

dataset_2 = sc.parallelize(["D", "E", "F"])

dataset_1.cartesian(dataset_2).collect()

[('A', 'D'), ('A', 'E'), ('A', 'F'), ('B', 'D'), ('B', 'E'), ('B', 'F')]

In [10]:
# sum, mean axamples

dataset_1 = sc.parallelize([1, 2, 3, 4, 5, 6])
(dataset_1.sum(), dataset_1.mean())

(21, 3.5)

In [11]:
# stats example

dataset_1.stats()

(count: 6, mean: 3.5, stdev: 1.707825127659933, max: 6.0, min: 1.0)

### Fundamendal RDD Functions on Tuples

* Recall that tuples of key, values are fundamental concept in the Map Reduce framework.
  * ex. `[("THE", 12), ("HI", "2"), ("COURSE", 2), ("STUDENTS", 3), ... ]`
* `Spark` provides a set of methods (transformations and actions) that can be applied to data tuples 

* Transformations:
  * `sortByKey`: returns a new RDD sorted by key   
  * `reduceByKey`: takes a function `f` as input and returns a new RDD where the values were reduced using `f`   
  * `groupByKey`: returns a new RDD where values were grouped by key
  * `join`: returns a new RDD by grouping values between keys in two datasets
    * Includes leftOuter, rightOuter, fullOuter

* Actions: 
  * Type agnostic actions such as `count` work on any type of data
  * Tuple RDD can be transformed using `map` before invoking an arithmetic action. E.g.:
  
```
data_1  = sc.parallelize([("A", 1), ("A", 3), ("B", 4), ("C", 6), ])
data_1.map(lambda x: x[1]).sum()  
```

In [12]:
# sortByKey example

data_1  = sc.parallelize([("C", 12), ("D", "2"), ("A", 2), ("B", 3)])
data_1.sortByKey().collect()


[('A', 2), ('B', 3), ('C', 12), ('D', '2')]

In [13]:
# reduceByKey example

data_1  = sc.parallelize([("A", 12), ("A", 2), ("C", 2), ("B", 3), ("C", 3)])
data_1.reduceByKey(lambda x,y: x+y).collect()


[('A', 14), ('B', 3), ('C', 5)]

In [14]:
# groupByKey example

data_1  = sc.parallelize([("A", 12), ("A", 2), ("C", 2), ("B", 3), ("C", 3), ("A", 5)])
grouped_data = data_1.groupByKey().collect()

for key, val in grouped_data:
    print(f"{key}\t{list(val)}")


A	[12, 2, 5]
B	[3]
C	[2, 3]


In [15]:
# join exmaple

data_1  = sc.parallelize([("A", 1), ("A", 3), ("B", 4), ("C", 6), ])
data_2  = sc.parallelize([("A", 2),           ("B", 5), ("C", 7), ])

data_1.join(data_2).collect()

[('B', (4, 5)), ('C', (6, 7)), ('A', (1, 2)), ('A', (3, 2))]

In [16]:
# leftOuterJoin exmaple

data_1  = sc.parallelize([("A", 1), ("D", 3), ("B", 4), ("C", 6), ])
data_2  = sc.parallelize([("A", 2),           ("B", 5), ("C", 7), ])

data_1.leftOuterJoin(data_2).collect()

[('B', (4, 5)), ('C', (6, 7)), ('A', (1, 2)), ('D', (3, None))]

In [17]:
# rightOuterJoin exmaple

data_1  = sc.parallelize([("A", 1), ("D", 3), ("B", 4), ("C", 6), ])
data_2  = sc.parallelize([("A", 2),           ("B", 5), ("C", 7), ])

data_1.rightOuterJoin(data_2).collect()

[('B', (4, 5)), ('C', (6, 7)), ('A', (1, 2))]

In [18]:
# apply non tuple actions  by first using map
# extract the values

data_1  = sc.parallelize([("A", 1), ("A", 3), ("B", 4), ("C", 6), ])
data_1.map(lambda x: x[1]).sum()

14

### Conslusion

* RDD is an immutable distributed collection of objects of your data
  * Lines, tuples, JSON objects, etc.
    
* RDD are partitioned across nodes and are operated in parallel with a low-level API
  * transformations on RDD generate new RDD

    
* There are many RDD methods; use `dir` to see which methods are available


* Note that RDD does not enforce structure
  * There is nothing that prevents something like 
    
    ```sc.parallelize([("A", 1), {"First": "John", "Salary": 125_000}, ("B", 4), ("C", 6), ])```
    
* Spark provides an data structure that imposes structure and validation on data
  * Allows for a better optimization of the data

### Spark DataFrame

* pySpark DataFrames are immutable distributed collection of data organized into named columns

  * Conceptually equivalent to a table in a relational database or Python DataFrame 
  * Supports SQL queries or Python like selects 

```
session.sql("SELECT * from users where age < 21")
```

or

```
users.filter(users.age < 21)
```

* The fact that the data is structured allows for richer optimizations under the hood

* DataFrames can be constructed from a wide array of sources such as: 
  * structured data files
  * tables in Hive 
  * external databases
  * or existing RDDs

* Just like RDDs were collections of objects, DataFrames are collections of instances (rows)


### Spark Functions on DataFrames

* Spark provides dozens of other high level functions that apply to its DataFrames
  * These functions use the map-reduce paradigm to implement useful and commonly-used functions
  * Important to remember that these are just conveniences that are efficiently implemented using the same core set of function: map, flatmap, filter and reduce
  
*  `SparkContext` was required to work with `RDD`s. Working DataFrames requires `SparkSession`
  * Create, register and execute SQL queries require `SparkSession`
  
* We can create a DataFrame (DF) using different methods
  * Reading a `csv`: every row is an object
  * Reading a `json`: every record is an object
    * Each line must contain a separate, self-contained valid JSON object.
      * This is a special format called newline-delimited JSON.
  * Reading a text file: every row is an object
  
    

### On Schemas

* Schemas describe the data types of your fields
* Schemas are at the heart of optimizations in Spark
* Schemas are critical for deriving the correct in-memory compression
  * After compression, data in RAM can be smaller than the raw data on disk
  


In [ ]:
from pyspark import SparkContext
sc = SparkContext()
from pyspark.sql import SparkSession
session = SparkSession(sc)

In [20]:
text_df  = session.read.text('./data/pride_and_prejudice.txt')
print(text_df.count())
text_df.first()

14579


Row(value='The Project Gutenberg eBook of Pride and Prejudice, by Jane Austen')

In [ ]:
%%time

csv_df = session.read.csv("data/nyc-flights.csv", header=True)
print(csv_df.count())
csv_df.head(1)


In [22]:
csv_df.schema

StructType(List(StructField(year,StringType,true),StructField(month,StringType,true),StructField(day,StringType,true),StructField(dep_time,StringType,true),StructField(dep_delay,StringType,true),StructField(arr_time,StringType,true),StructField(arr_delay,StringType,true),StructField(carrier,StringType,true),StructField(tailnum,StringType,true),StructField(flight,StringType,true),StructField(origin,StringType,true),StructField(dest,StringType,true),StructField(air_time,StringType,true),StructField(distance,StringType,true),StructField(hour,StringType,true),StructField(minute,StringType,true)))

In [23]:
StructType(
	List(StructField(year,StringType,true),
		StructField(month,StringType,true),
		StructField(day,StringType,true),
		StructField(dep_time,StringType,true),
		StructField(dep_delay,StringType,true),
		StructField(arr_time,StringType,true),
		StructField(arr_delay,StringType,true),
		StructField(carrier,StringType,true),
		StructField(tailnum,StringType,true),
		StructField(flight,StringType,true),
		StructField(origin,StringType,true),
		StructField(dest,StringType,true),
		StructField(air_time,StringType,true),
		StructField(distance,StringType,true),
		StructField(hour,StringType,true),
		StructField(minute,StringType,true)
	)
)

NameError: name 'StructType' is not defined

In [24]:
%%time

csv_df = session.read.options(inferSchema = True).csv("data/nyc-flights.csv", header=True)

print(csv_df.count())

csv_df.head(2)


32735
CPU times: user 13.9 ms, sys: 8.83 ms, total: 22.7 ms
Wall time: 2.26 s


[Row(year=2013, month=6, day=30, dep_time=940, dep_delay=15, arr_time=1216, arr_delay=-4, carrier='VX', tailnum='N626VA', flight=407, origin='JFK', dest='LAX', air_time=313, distance=2475, hour=9, minute=40),
 Row(year=2013, month=5, day=7, dep_time=1657, dep_delay=-3, arr_time=2104, arr_delay=10, carrier='DL', tailnum='N3760C', flight=329, origin='JFK', dest='SJU', air_time=216, distance=1598, hour=16, minute=57)]

In [25]:
csv_df.schema

StructType(List(StructField(year,IntegerType,true),StructField(month,IntegerType,true),StructField(day,IntegerType,true),StructField(dep_time,IntegerType,true),StructField(dep_delay,IntegerType,true),StructField(arr_time,IntegerType,true),StructField(arr_delay,IntegerType,true),StructField(carrier,StringType,true),StructField(tailnum,StringType,true),StructField(flight,IntegerType,true),StructField(origin,StringType,true),StructField(dest,StringType,true),StructField(air_time,IntegerType,true),StructField(distance,IntegerType,true),StructField(hour,IntegerType,true),StructField(minute,IntegerType,true)))

```
StructType(
	List(
		StructField(year,IntegerType,true),
		StructField(month,IntegerType,true),
		StructField(day,IntegerType,true),
		StructField(dep_time,IntegerType,true),
		StructField(dep_delay,IntegerType,true),
		StructField(arr_time,IntegerType,true),
		StructField(arr_delay,IntegerType,true),
		StructField(carrier,StringType,true),
		StructField(tailnum,StringType,true),
		StructField(flight,IntegerType,true),
		StructField(origin,StringType,true),
		StructField(dest,StringType,true),
		StructField(air_time,IntegerType,true),
		StructField(distance,IntegerType,true),
		StructField(hour,IntegerType,true),
		StructField(minute,IntegerType,true)
	)
)
```

In [26]:
json_df  = session.read.json('./data/random_user_dicts.json')
print(json_df.count())
json_df.printSchema()


5000
root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- lat_long: struct (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |-- state: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- zip: long (nullable = true)



In [27]:
json_df.show()

+-----------+---------+--------------------+------------+-----------+-----+
| first_name|last_name|            lat_long|       state|    user_id|  zip|
+-----------+---------+--------------------+------------+-----------+-----+
|Christopher|   Morgan|  {6.4442, -78.5063}|    Nebraska|895-76-0473|73093|
|      Riley| Franklin|  {86.3108, 81.9157}|     Indiana|218-53-2453|43053|
|      Tammy|Gutierrez|{10.2281, -141.5519}|    Delaware|290-41-4495|11862|
|       Toni|  Steward| {84.6309, -97.4512}|     Alabama|937-19-1777|89473|
|    Vanessa|   Vargas|{-46.5554, -22.8648}|    New York|036-93-1373|64349|
|   Brooklyn|   Parker| {15.5293, 158.2673}|    Arkansas|946-76-3712|83694|
|       Andy|  Carroll| {-42.0753, 91.2244}|     Wyoming|350-48-6550|74170|
|     Austin|    Gomez|  {0.0292, -15.2938}|  Washington|491-00-8790|35510|
|     Ernest|   Barnes|{-21.7537, -175.5...|    New York|990-89-5629|40333|
|    Tiffany|  Bennett|{-42.3378, -68.4757}|    Colorado|624-71-6721|91276|
|      Grace

In [28]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType

json_struct = StructType([
    StructField("first_name", StringType(), nullable=False, metadata=None),
    StructField("last_name", StringType(),  nullable=False, metadata=None),
    StructField("lat_long", 
                StructType([
                    StructField("latitude", FloatType(), metadata=None, nullable=True),
                    StructField("longitude", FloatType(), metadata=None, nullable=True)
                ]), nullable=True, metadata=None),
    StructField("state", StringType(),  nullable=True, metadata=None),
    StructField("user_id", StringType(),  nullable=True, metadata=None),
    StructField("zip", StringType(),  nullable=True, metadata=None),    
])


In [29]:
import pyspark
dir(pyspark.sql.types)

['ArrayType',
 'AtomicType',
 'BinaryType',
 'BooleanType',
 'ByteType',
 'CloudPickleSerializer',
 'DataType',
 'DataTypeSingleton',
 'DateConverter',
 'DateType',
 'DatetimeConverter',
 'DecimalType',
 'DoubleType',
 'FloatType',
 'FractionalType',
 'IntegerType',
 'IntegralType',
 'JavaClass',
 'LongType',
 'MapType',
 'NullType',
 'NumericType',
 'Row',
 'ShortType',
 'SparkContext',
 'StringType',
 'StructField',
 'StructType',
 'TimestampType',
 'UserDefinedType',
 '_FIXED_DECIMAL',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_acceptable_types',
 '_all_atomic_types',
 '_all_complex_types',
 '_array_signed_int_typecode_ctype_mappings',
 '_array_type_mappings',
 '_array_unsigned_int_typecode_ctype_mappings',
 '_atomic_types',
 '_create_converter',
 '_create_row',
 '_create_row_inbound_converter',
 '_has_nulltype',
 '_infer_schema',
 '_infer_type',
 '_int_size_to_type',
 '_make_type_verifier',
 '_mer

In [31]:
import pprint
import json

json_df  = session.read.schema(json_struct).json('./data/random_user_dicts.json')
print(json_df.count())
new_json = json_df.schema.json()
pprint.pprint(json.loads(new_json))

5000
{'fields': [{'metadata': {},
             'name': 'first_name',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'last_name',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'lat_long',
             'nullable': True,
             'type': {'fields': [{'metadata': {},
                                  'name': 'latitude',
                                  'nullable': True,
                                  'type': 'float'},
                                 {'metadata': {},
                                  'name': 'longitude',
                                  'nullable': True,
                                  'type': 'float'}],
                      'type': 'struct'}},
            {'metadata': {},
             'name': 'state',
             'nullable': True,
             'type': 'string'},
            {'metadata': {},
             'name': 'user_id',
      

In [32]:
# Note the lat_long field. It has its own format

json_df.head(1)

[Row(first_name='Christopher', last_name='Morgan', lat_long=Row(latitude=6.444200038909912, longitude=-78.50630187988281), state='Nebraska', user_id='895-76-0473', zip='73093')]

### On Data Formats

* Structured data (ex. tab or comma delimited tables) are ideal when data does not changes
    * This is not the case in real life, data changes    

* In Semi-structured data, fields are not necessarily shared by all observations

```
[ {"user_id": "Jane1234", employed: True, "salary": "95000"}, 
  {"user_id": "Johhn777", employed: False}, 
  ...
]
```


* Also, tables do not allow for nested structures
  * Cars is a list
  * Each child has its own nested structure
[..., {
        "user_id": "Jane1234", 
        "cars": ["Sedan", "Truck"]
        "children": {
        "Jonah": {"age": 8
                  "School": "Noelani Elementary"},
        "Mary": {"age": 12
                  "School": "Sacred Hearts"}
                  
        } 
      }...]


* Disadvantages of JSON here:
  * Space on disk (duplicated fields)
    * Fields are not stored in the objects (in RAM) 
  * JSON, don't work with tranditional SQL
    * We can do programmatic queries
    * We can creare tempViews that can be queried 
      * Spark knows how to convert a colleciton of JSON objects to a table


### On DataFrames and SQL

* DataFrames support SQL 
  * SQL Only supported on DataFrames
* The same SQL queries work out of the box on DataFrames
* Use the `.sql()` method to pass it a SQL query
  * Need to create a view
* Queries are evaluated for optimization prior to execution 
  * Standard SQL-type query optimizations        
* Optimization is done using Catalyst SQL optimizer 
    https://databricks.com/glossary/catalyst-optimizer
* Helps to remember that the data is immutable
    * adding new colums needs to be done on a new DataFrame


# Example of query optimization
![](https://www.dropbox.com/s/e756fxrsi36yvj4/unoptimized_optimized.png?dl=1)
    

In [57]:
json_df  = session.read.schema(json_struct).json('./data/random_user_dicts.json')
print(json_df.count())


5000


In [303]:
json_df.createTempView("users")

session.sql("""
SELECT first_name, COUNT(*)
FROM users
GROUP BY first_name; 
""").show()

+----------+--------+
|first_name|count(1)|
+----------+--------+
|     Tyler|       6|
|  Samantha|      10|
|    Aubrey|       4|
|   Carolyn|      11|
|      Chad|       9|
|   Shannon|       8|
|     Shawn|       5|
|       Sue|      11|
|     Scott|       6|
|     Ruben|       9|
|     Flenn|       5|
|  Rosemary|       3|
|     Grace|       7|
|     Lucas|       8|
|     Keith|      12|
|    Gerald|      10|
|       Jar|       6|
|     Edwin|       6|
|     Soham|       5|
|  Savannah|       7|
+----------+--------+
only showing top 20 rows



In [269]:
session.sql("""
SELECT *
FROM users
WHERE first_name IN ("Evan", "Sarah", "John"); 
""").show()

+----------+---------+--------------------+-------------+-----------+-----+
|first_name|last_name|            lat_long|        state|    user_id|  zip|
+----------+---------+--------------------+-------------+-----------+-----+
|      Evan|     Beck|  {-17.7362, -3.022}|New Hampshire|505-92-9095|87501|
|      Evan|   Snyder| {56.7583, -64.3217}|    Minnesota|075-11-1233|74201|
|     Sarah|     Webb|   {2.5834, -89.618}| South Dakota|247-48-1845|86063|
|     Sarah|  Stanley|{-18.5369, -81.8778}|     Missouri|997-11-7309|68852|
|      John|  Nichols|{-10.576, -148.6093}|      Alabama|575-68-2404|17965|
|      John|     Reid|   {0.659, -70.5511}|  Mississippi|370-33-2662|57788|
|      John|  Freeman|{-69.3141, -12.2351}|        Maine|621-84-6581|13221|
|      John|    Price| {-7.5446, 109.4057}| North Dakota|323-62-2196|18403|
|      John|    Davis|{-39.9862, -72.4857}|        Maine|211-00-2584|56206|
|      Evan|   Torres|{-81.3332, 118.9237}|      Vermont|547-50-2905|64904|
|      Evan|

### DataFrame and Select Queries

* The same functionality is available using Python
* Many additional functions, inlcuding analytics-specific ones are available through specific library

    ```from pyspark.sql import functions as F```
    * The functions as used with a select
  * Can use `agg` to do specific operations and rename columns.

In [291]:
json_df.filter(F.length(json_df.first_name) < 4).show()

+----------+---------+--------------------+-------------+-----------+-----+
|first_name|last_name|            lat_long|        state|    user_id|  zip|
+----------+---------+--------------------+-------------+-----------+-----+
|       Joe|  Jackson|{-62.895, -143.5974}|      Florida|664-45-7303|23155|
|       Mia|   Chavez|  {13.112, 109.6125}|     Arkansas|309-47-7003|42409|
|       Jon|     Cole| {71.6651, 101.2318}| Rhode Island|217-81-4486|92912|
|       Mia|  Barrett| {-57.829, -63.2612}|New Hampshire|236-16-5120|31229|
|       Max|   Willis|   {31.288, 52.5112}|         Utah|858-00-9946|78049|
|       Kim|  Spencer|  {50.676, -70.7382}|        Idaho|715-58-2909|67867|
|       Roy| Mitchell|   {49.4706, 1.6422}|   Washington|421-56-4226|62647|
|       Eli|   Turner|{-42.0611, 144.1344}|     Delaware|744-31-7784|96253|
|       Ida|  Gilbert|   {1.7455, 81.5158}|    Louisiana|777-95-8206|65696|
|       Joe|      Fox| {-57.2874, 25.6431}|      Montana|754-89-1224|18754|
|       Lee|

In [295]:
json_df.groupby("first_name").count().show()

+---------+-----+
|last_name|count|
+---------+-----+
| Harrison|   14|
|   Porter|   17|
|    Scott|   23|
|Robertson|   20|
|   Wilson|   17|
|  Griffin|   15|
|    Lucas|   22|
|   Castro|   13|
|     Pena|   13|
|     Boyd|   25|
|    Jones|   18|
|   Graham|   18|
|  Herrera|   17|
| Crawford|   24|
|     Lowe|   11|
|  Sanchez|   13|
|Gutierrez|   15|
|    Garza|   18|
|    James|   15|
|     Soto|   13|
+---------+-----+
only showing top 20 rows



In [271]:
from pysparak.sql import functions as F

dir(F)

['Column',
 'DataFrame',
 'DataType',
 'PandasUDFType',
 'PythonEvalType',
 'SparkContext',
 'StringType',
 'UserDefinedFunction',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_udf',
 '_get_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_column',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aggregate',
 'approxCountDistinct',
 'approx_count_distinct',
 'array',
 'array_contains',
 'array_distinct',
 'array_except',
 'array_intersect',
 'array_join',
 'array_max',
 'array_min',
 'array_position',
 'array_remove',
 'array_repeat',
 'array_sort',
 'array_union',
 'arrays_overlap',
 'arrays_zip',
 'asc',
 'asc_nulls_first',
 'asc_nulls_last',
 'ascii',
 'asi

In [281]:
json_df.groupby("first_name").agg(F.count("first_name").alias('First Name Counts')).show()

+----------+-----------------+
|first_name|First Name Counts|
+----------+-----------------+
|     Tyler|                6|
|  Samantha|               10|
|    Aubrey|                4|
|   Carolyn|               11|
|      Chad|                9|
|   Shannon|                8|
|     Shawn|                5|
|       Sue|               11|
|     Scott|                6|
|     Ruben|                9|
|     Flenn|                5|
|  Rosemary|                3|
|     Grace|                7|
|     Lucas|                8|
|     Keith|               12|
|    Gerald|               10|
|       Jar|                6|
|     Edwin|                6|
|     Soham|                5|
|  Savannah|                7|
+----------+-----------------+
only showing top 20 rows



### More on Optimization

1. Caching the data
  * Caches contents of a table or output of a query
  * Cached tables can occupy substantially less RAM that the version on disk
    * Tungsten compression
  * Can implement Lazy Caching to cache as needed
    * UNCACHE to leverage space for caching other Data Frames
2. Changing the number of data frame partition
  * Some operations are self-contained to a partition: Narrow operation
    * example, `COUNT` can operate on a single partition
  * Some operations require data from other partitions: Wide operation
    * example, `GROUPBY ` requires transferring a partition to another node so that data two partitions are grouped
  * If the number of "resulting" partitions is large, the bulk of the operation time will be spent on data transfer 

* Only scratches the surface of performance tuning:
  * See here for more: https://spark.apache.org/docs/latest/sql-performance-tuning.html     

In [58]:
!ls -l --block-size=M ./data/random_user_dicts_larger.json

-rw-r--r-- 1 jovyan users 798M Sep 22 20:54 ./data/random_user_dicts_larger.json


In [59]:
json_df_large  = session.read.option("inferSchema", "true").json('./data/random_user_dicts_large.json')
print(json_df_large.count())


500000


In [60]:
session.catalog.dropTempView("users_large")

In [61]:
json_df_large.createTempView("users_large")
#session.catalog.dropTempView("users_large")

temp_df = session.sql("""
SELECT first_name, COUNT(*)
FROM users_large
GROUP BY first_name; 
""")

temp_df.explain(mode="formatted")

== Physical Plan ==
* HashAggregate (4)
+- Exchange (3)
   +- * HashAggregate (2)
      +- Scan json  (1)


(1) Scan json 
Output [1]: [first_name#1325]
Batched: false
Location: InMemoryFileIndex [file:/home/jovyan/work/data/random_user_dicts_large.json]
ReadSchema: struct<first_name:string>

(2) HashAggregate [codegen id : 1]
Input [1]: [first_name#1325]
Keys [1]: [first_name#1325]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#1352L]
Results [2]: [first_name#1325, count#1353L]

(3) Exchange
Input [2]: [first_name#1325, count#1353L]
Arguments: hashpartitioning(first_name#1325, 200), ENSURE_REQUIREMENTS, [id=#813]

(4) HashAggregate [codegen id : 2]
Input [2]: [first_name#1325, count#1353L]
Keys [1]: [first_name#1325]
Functions [1]: [count(1)]
Aggregate Attributes [1]: [count(1)#1348L]
Results [2]: [first_name#1325, count(1)#1348L AS count(1)#1349L]




In [62]:
json_df_large.rdd.getNumPartitions()

4

In [63]:
temp_df.rdd.getNumPartitions()

200

In [64]:
%%timeit
temp_df.count()

1.59 s ± 183 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
temp_df_2 = temp_df.coalesce(3)
temp_df_2.rdd.getNumPartitions()

3

In [66]:
%%timeit
temp_df_2.count()

778 ms ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
%%time
json_df_larger  = session.read.option("inferSchema", "true").json('./data/random_user_dicts_larger.json')
print(json_df_larger.count())


5000000
CPU times: user 9.63 ms, sys: 7.98 ms, total: 17.6 ms
Wall time: 7.19 s


In [77]:
%%time

# inferring schema can be a computationally expensive operation on large data
json_df_larger  = session.read.json('./data/random_user_dicts_larger.json')


CPU times: user 3.82 ms, sys: 7.32 ms, total: 11.1 ms
Wall time: 4.93 s


In [78]:
session.catalog.dropTempView("users_larger")
json_df_larger.createTempView("users_larger")

In [79]:
%%timeit
session.sql("""
SELECT COUNT(*) FROM users_larger
""").collect()

2.48 s ± 68.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [81]:
%%time

session.sql("""
SELECT COUNT(*) FROM users_larger
""").collect()

CPU times: user 10.3 ms, sys: 1.12 ms, total: 11.4 ms
Wall time: 2.64 s


[Row(count(1)=5000000)]

In [82]:
%%time
session.sql("""
CACHE TABLE users_larger
""").collect()

### Should be viewable on SparkUI

CPU times: user 10.3 ms, sys: 5.3 ms, total: 15.6 ms
Wall time: 17.2 s


[]

In [83]:
%%timeit
session.sql("""
SELECT COUNT(*) FROM users_larger
""").collect()

73.5 ms ± 3.86 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
session.sql("""
UNCACHE TABLE users_larger
""").collect()

[]

In [85]:
%%time
session.sql("""
SELECT COUNT(*) FROM users_larger
""").collect()

CPU times: user 2.73 ms, sys: 7.77 ms, total: 10.5 ms
Wall time: 2.64 s


[Row(count(1)=5000000)]